Check out the example_data folder for an example on how to prepare dataset

In [3]:
# set folder path where your data is stored
data_folder = '/content/tickets_pick'

Code to create a csv file with a list of all the files present in your data folder

In [4]:
import os
import pandas
import json
import csv
import shutil

data_type = 'ticket' # enter your document type e.g. receipt, ticket, forms etc.
train_samples_list =  []
for file in os.listdir(data_folder+'/box'):
  train_samples_list.append([data_type,file.replace('.csv','')])
train_samples_list = pandas.DataFrame(train_samples_list)
train_samples_list.to_csv("samples_list.csv")

Clone repository

In [5]:
!git clone https://github.com/sharmas1ddharth/PICK_pytorch.git

Cloning into 'PICK_pytorch'...
remote: Enumerating objects: 288, done.
remote: Counting objects: 100% (288/288), done.
remote: Compressing objects: 100% (257/257), done.
remote: Total 288 (delta 26), reused 286 (delta 26), pack-reused 0
Receiving objects: 100% (288/288), 17.24 MiB | 29.23 MiB/s, done.
Resolving deltas: 100% (26/26), done.


Change current working directory

In [6]:
%cd '/content/PICK_pytorch'

/content/PICK_pytorch


In [ ]:
## Installing requirements for running PICK-pytorch
!pip install -r requirements.txt
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Divide samples_list.csv into train set and test set

In [9]:
from sklearn.model_selection import train_test_split
train_test = pandas.read_csv("/content/samples_list.csv",dtype=str)
train, test= train_test_split(train_test,test_size=0.1,random_state = 42)

Copy images, boxes and entities from data folder to train folder inside pick_pytorch and create a train_samples_list.csv according to the train set

In [10]:
for index, row in train.iterrows():
  shutil.copy(data_folder+'/box/'+str(row[2]),'/content/PICK_pytorch/data/train_data/boxes_and_transcripts')
  shutil.copy(data_folder+'/img/'+str(row[2]).replace(".tsv",".jpg"),'/content/PICK_pytorch/data/train_data/images')
  shutil.copy(data_folder+'/entities/'+str(row[2]).replace(".tsv",".txt"), '/content/PICK_pytorch/data/train_data/entities')

train.drop(['Unnamed: 0'], axis = 1,inplace = True)
train.reset_index(inplace= True)
train.drop(['index'], axis = 1,inplace = True)
train.to_csv("/content/PICK_pytorch/data/train_data/train_samples_list.csv",header = False)

Copy images, boxes and entities from data folder to test folder inside pick_pytorch and create a test_samples_list.csv according to the test set

In [11]:
for index, row in test.iterrows():
  shutil.copy(data_folder+'/box/'+str(row[2]),'/content/PICK_pytorch/data/val_data/boxes_and_transcripts')
  shutil.copy(data_folder+'/img/'+str(row[2]).replace(".tsv",".jpg"),'/content/PICK_pytorch/data/val_data/images')

test.drop(['Unnamed: 0'], axis = 1,inplace = True)
test.reset_index(inplace= True)
test.drop(['index'], axis = 1,inplace = True)
test.to_csv("/content/PICK_pytorch/data/val_data/test_samples_list.csv",header = False)

Create a list of your entities and write to entites_list.py 

In [12]:
%%writefile utils/entities_list.py
Entities_list = [
"total",
"source",
"destination",
"departure",
"arrival",
"train_name",
"pnr",
"gst_no",
]

Overwriting utils/entities_list.py


Start training

In [ ]:
#!/bin/bash
!python -m torch.distributed.launch --nnode=1 --node_rank=0 --nproc_per_node=1 \
   train.py -c config.json -d 0 --local_world_size 1
  # --resume /content/PICK-pytorch/saved/models/PICK_Default/test_0917_074722/model_best.pth ##uncomment for resume training

[2022-05-13 11:43:57,149 - train - INFO] - Distributed GPU training model start...
[2022-05-13 11:43:57,149 - train - INFO] - [Process 470] Initializing process group with: {'MASTER_ADDR': '127.0.0.1', 'MASTER_PORT': '29500', 'RANK': '0', 'WORLD_SIZE': '1'}
[2022-05-13 11:43:57,150 - train - INFO] - [Process 470] world_size = 1, rank = 0, backend=nccl
[2022-05-13 11:43:57,158 - train - INFO] - Dataloader instances created. Train datasets: 63 samples Validation datasets: 8 samples.
[2022-05-13 11:43:57,906 - train - INFO] - Model created, trainable parameters: 65358434.
[2022-05-13 11:43:57,907 - train - INFO] - Optimizer and lr_scheduler created.
[2022-05-13 11:43:57,907 - train - INFO] - Max_epochs: 150 Log_per_step: 10 Validation_per_step: 50.
[2022-05-13 11:43:57,907 - train - INFO] - Training start...
[2022-05-13 11:43:57,911 - trainer - INFO] - [Process 470] world_size = 1, rank = 0, n_gpu/process = 1, device_ids = [0]
/usr/local/lib/python3.7/dist-packages/torch/autograd/anomaly_

## Testing

In [ ]:
import os

os.makedirs('/content/test_box', exist_ok=True)
os.makedirs('/content/test_images', exist_ok=True)
os.makedirs('/content/output', exist_ok=True)

test_box_path = '/content/test_box'
test_images_path = '/content/test_images'
output_path = '/content/output'

model_path = '' # paste your model path here. It should be inside PICK_pytorch/saved/models/PICK_Default/

In [ ]:
## change model_best.pth path
!python test.py --checkpoint {model_path} \
                --boxes_transcripts {test_box_path} \
                --images_path {test_img_path} --output_folder {output_path} \
                --gpu 0 --batch_size 1

Loading checkpoint: /content/Pick.Pytorch.Sroie/saved/models/PICK_Default/test_0508_214546/checkpoint-epoch20.pth 
with saved mEF 0.4982 ...
boxes_and_transcripts_folder /content/test_boxes_and_transcripts/
9it [00:06,  1.36it/s]


You can see predictions in output folder.

It will look something like this

```
company	ADVANCO COMPANY,co
address	NO 1&3\, JALAN ANGSA DELIMA 12
address	WANGSA LINK\, WANGSA MAJU
address	53300 KUALA LUMPUR
date	23/03/2018
```



